In [192]:
import pandas as ps
import numpy as np
import re
import csv

In [193]:
def openFile(path):
    return ps.read_csv(path, index_col=0) #index_col=0 has the effect of excluding any unnamed fields

def toDF(rawData):
    return ps.DataFrame(rawData)

def dropDuplicates(rawData):
    return toDF(rawData).drop_duplicates()

def filter(df, columns):
    return df[columns] 

def merge(df1, df2, protocol, on):
    return ps.merge(df1, df2, how=protocol, on=on)

def extractYear(title):
    li = re.search('(\d{4})', title)
    if li:
        return li[1]
    return -1

In [194]:
# Drop duplicates
rawDF = dropDuplicates(openFile('using/winemag-data-130k-v2.csv'))
print(filter(rawDF, ['title']))

                                                    title
id                                                       
0                       Nicosia 2013 Vulkà Bianco  (Etna)
1           Quinta dos Avidagos 2011 Avidagos Red (Douro)
2           Rainstorm 2013 Pinot Gris (Willamette Valley)
3       St. Julian 2013 Reserve Late Harvest Riesling ...
4       Sweet Cheeks 2012 Vintner's Reserve Wild Child...
...                                                   ...
129966  Dr. H. Thanisch (Erben Müller-Burggraef) 2013 ...
129967                  Citation 2004 Pinot Noir (Oregon)
129968  Domaine Gresser 2013 Kritt Gewurztraminer (Als...
129969      Domaine Marcel Deiss 2012 Pinot Gris (Alsace)
129970  Domaine Schoffit 2012 Lieu-dit Harth Cuvée Car...

[119988 rows x 1 columns]


In [195]:
# Extract year from dataset
with open('using/years.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["id","year"])
    for idx,row in enumerate(filter(rawDF, ['title']).itertuples(index=True, name='Pandas')):
        writer.writerow([idx,extractYear(row.title)])

newDF = openFile('using/years.csv')
print(newDF)

        year
id          
0       2013
1       2011
2       2013
3       2013
4       2012
...      ...
119983  2013
119984  2004
119985  2013
119986  2012
119987  2012

[119988 rows x 1 columns]


In [196]:
# Merge the two datasets
merge(rawDF, newDF, 'outer', "id").to_csv('using/cleaned.csv')
print(openFile('using/cleaned.csv'))

         country                                        description  \
id                                                                    
0          Italy  Aromas include tropical fruit, broom, brimston...   
1       Portugal  This is ripe and fruity, a wine that is smooth...   
2             US  Tart and snappy, the flavors of lime flesh and...   
3             US  Pineapple rind, lemon pith and orange blossom ...   
4             US  Much like the regular bottling from 2012, this...   
...          ...                                                ...   
119977       NaN                                                NaN   
119978       NaN                                                NaN   
119979       NaN                                                NaN   
119980       NaN                                                NaN   
119981       NaN                                                NaN   

                               designation  points  price           province

Using price, points, variety, region1, country, winery, year. Drop everything else.

In [197]:
# Filter for interested fields
df = filter(toDF(openFile('using/cleaned.csv')), ["title","price", "points", "variety", "region_1", "country", "winery", "year"])
print(df)

                                                    title  price  points  \
id                                                                         
0                       Nicosia 2013 Vulkà Bianco  (Etna)    NaN    87.0   
1           Quinta dos Avidagos 2011 Avidagos Red (Douro)   15.0    87.0   
2           Rainstorm 2013 Pinot Gris (Willamette Valley)   14.0    87.0   
3       St. Julian 2013 Reserve Late Harvest Riesling ...   13.0    87.0   
4       Sweet Cheeks 2012 Vintner's Reserve Wild Child...   65.0    87.0   
...                                                   ...    ...     ...   
119977                                                NaN    NaN     NaN   
119978                                                NaN    NaN     NaN   
119979                                                NaN    NaN     NaN   
119980                                                NaN    NaN     NaN   
119981                                                NaN    NaN     NaN   

           

In [200]:
# Remove any obviously corrupted entries
drop = []
for idx,row in enumerate(df.itertuples(index=True, name='Pandas')):
    for ele in row:
        if ps.isna(ele):
            drop.append(idx)
# Drop the marked rows
df.drop(df.index[drop]).to_csv('using/cleaned.csv')
# Reopen cleaned file
df = toDF(openFile('using/cleaned.csv'))
print(df)

                                                    title  price  points  \
id                                                                         
2           Rainstorm 2013 Pinot Gris (Willamette Valley)   14.0    87.0   
3       St. Julian 2013 Reserve Late Harvest Riesling ...   13.0    87.0   
4       Sweet Cheeks 2012 Vintner's Reserve Wild Child...   65.0    87.0   
5       Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...   15.0    87.0   
6        Terre di Giurfo 2013 Belsito Frappato (Vittoria)   16.0    87.0   
...                                                   ...    ...     ...   
119982           El Enemigo 2014 Cabernet Franc (Mendoza)   28.0    90.0   
119983       Frank Family 2013 Petite Sirah (Napa Valley)   35.0    90.0   
119984  Giacosa Fratelli 2013 Basarin Vigna Gianmatè  ...   55.0    90.0   
119986  VGS Chateau Potelle 2015 Eau de Tony Sauvignon...   40.0    90.0   
119987  Vinessens 2014 La Casica del Abuelo Red (Alica...   42.0    90.0   

           

In [201]:
# Remove any negative number entries and such
"""
row[3] == price
row[4] == points
"""
drop = []
for idx,row in enumerate(df.itertuples(index=True, name='Pandas')):
    if row[2] < 0 or row[3] < 0:
        drop.append(idx)
df.drop(df.index[drop]).to_csv('using/cleaned.csv')
# Reopen cleaned file
df = toDF(openFile('using/cleaned.csv'))
print(df)

                                                    title  price  points  \
id                                                                         
2           Rainstorm 2013 Pinot Gris (Willamette Valley)   14.0    87.0   
3       St. Julian 2013 Reserve Late Harvest Riesling ...   13.0    87.0   
4       Sweet Cheeks 2012 Vintner's Reserve Wild Child...   65.0    87.0   
5       Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...   15.0    87.0   
6        Terre di Giurfo 2013 Belsito Frappato (Vittoria)   16.0    87.0   
...                                                   ...    ...     ...   
119982           El Enemigo 2014 Cabernet Franc (Mendoza)   28.0    90.0   
119983       Frank Family 2013 Petite Sirah (Napa Valley)   35.0    90.0   
119984  Giacosa Fratelli 2013 Basarin Vigna Gianmatè  ...   55.0    90.0   
119986  VGS Chateau Potelle 2015 Eau de Tony Sauvignon...   40.0    90.0   
119987  Vinessens 2014 La Casica del Abuelo Red (Alica...   42.0    90.0   

           

Cleaning checklist
- Dropped duplicates
- Added years to a cleaned csv
- Dropped unnecessary columns
- Remove any NaN rows
- Remove unnamed fields
- Drop any rows with negative price and points